In [1]:
code = {
    "name": "commands",
    "body": [
        {
            "name": "repeat-n",
            "count": 10,
            "body": [
                {
                    "name": "gpio-write",
                    "pin": 16,
                    "value": 1
                },
                {
                    "name": "wait-sec",
                    "duration": 1
                },
                {
                    "name": "gpio-write",
                    "pin": 16,
                    "value": 0
                },
                {
                    "name": "wait-sec",
                    "duration": 1
                }
            ]
        }
    ]
}

In [2]:
import parser.parser as parser


p = parser.parse(code)

#print(p)
print(p.pprint())

SeqExp([
  RepeatExp(10, [
    SeqExp([
      ProcSetExp(gpio_mode[16], ValExp(1))
      ProcSetExp(gpio_value[16], ValExp(1))
    ])
    WaitTimeExp(1)
    SeqExp([
      ProcSetExp(gpio_mode[16], ValExp(1))
      ProcSetExp(gpio_value[16], ValExp(0))
    ])
    WaitTimeExp(1)
  ])
])
